In [124]:
import pandas as pd
import string

# 段落切分函數

In [125]:
def split_x(x,n=20):
    results = []
    if '+1)' in x:
        for i in range(1,n):
            text = x[x.find(f'+{i})'):x.find(f'+{i+1})')]
            if len(text) > 5:
                results.append(str(text))
    
    elif '1)' in x:
        for i in range(1,n):
            text = x[x.find(f'{i})'):x.find(f'{i+1})')]
            if len(text) > 5:
                results.append(str(text))
    
    elif '1.' in x:
        for i in range(1,n):
            text = x[x.find(f'{i}.'):x.find(f'{i+1}.')]
            if len(text) > 5:
                results.append(str(text))
    
    elif '+' in x:
        for text in x.split('+'):
            if len(text) > 5:
                results.append('+'+str(text))
    
    elif '(A)' in x:
        for i,j in zip(string.ascii_uppercase,string.ascii_uppercase[1:]+'A'):
            text = x[x.find(f'({i})'):x.find(f'({j})')]
            if len(text) > 5:
                results.append(str(text))
            
    return results

# 讀取押匯文件寶典

In [126]:
df = pd.read_excel('../data/20210831 押匯文件清單- 202007-202106.老師.xlsx')[['文件類別:','可能的說法']]
for i in df.columns:
    df[i] = df[i].apply(lambda x:x.replace('\n',''))
def string2list(s):
    return [ i.replace('\n','') for i in s.split('、')]
df['可能的說法'] = df['可能的說法'].apply(string2list)
def create_mapping(df):
    寶典 = {}
    for idx in df.index:
        d = {}
        for p in df.loc[idx,'可能的說法']:
            if p != '':
                d[p.strip()] = df.loc[idx,'文件類別:'].replace(' ','')
        寶典.update(d)
    return 寶典
寶典 = create_mapping(df)
寶典

{'BENEFICIARYS DRAFT': '匯票',
 'DRAFTS': '匯票',
 'BILL OF LADING': '提單',
 'TRANSPORT DOCUMENTS': '提單',
 'B/L': '提單',
 'MARINE/OCEAN BILL OF LADINGOCEANBILL OF LADING': '提單',
 'OCEAN BILL(S) OFLADING': '提單',
 'BILL(S) OF LADING': '提單',
 'AIR WAYBILLS': '提單',
 'COMMERCIAL INVOICE': '商業發票',
 'INVOICE': '商業發票',
 'INV.': '商業發票',
 'PACKING LIST': '裝箱單',
 'PACKING': '裝箱單',
 'P/L': '裝箱單',
 'INSURANCE POLICY/CERTIFICATE': '保險單據',
 'INSURANCE CERTIFICATE/POLICY': '保險單據',
 'I/PMARINE INSURANCE POLICY': '保險單據',
 'MARINE CARGO POLICY': '保險單據',
 'CARGO POLICY (INSURANCE AGENCY)': '保險單據',
 'INSURANCE DOCUMENT': '保險單據',
 'INSURANCE POLICY OR CERTIFICATE': '保險單據',
 'Insurance Certificate': '保險單據',
 'CONSULAR DECLARATION': '領事簽證(C/I)',
 'CONSULAR INVOICE': '領事簽證(C/I)',
 'CONSULAGE': '領事簽證(C/I)',
 'CONSULAR LEGALIZED INVOICE': '領事簽證(C/I)',
 'CERTIFICATE OF ORIGIN': '產地(商會)證明',
 'COOECFA DOCUMENT': '產地(商會)證明',
 'Certificate of Taiwan Origin': '產地(商會)證明',
 'CERTIFICATE OF TAIWANESE  ORIGIN': '產地(商會)證明',
 'TE

In [127]:
df = pd.read_excel('../data/combined_excel.xlsx',index_col=0)[['LCNO','46A','47A']].dropna(axis=0)
idx = df.sample(9).index
df.loc[idx]

,LCNO,46A,47A
20378,0AEOB20024601012,"+COMMERCIAL INVOICE IN 3 COPIES,SIGNED AND IND...",+5 PCT MORE OR LESS IN QUANTITY AND AMOUNT ARE...
17168,905IMS000086,+SIGNED COMMERCIAL INVOICE IN 4 COPIES INDICAT...,+ALL DOCUMENTS MUST BE ISSUED IN ENGLISH UNLES...
15300,OLC0015200037588,1.DRAFT FOR 100 PCT OF INVOICE VALUE..2.COMPLE...,1.IMPORT IS COVERED UNDER NON NEGATIVE LIST PO...
3963,LC1903021000108,1. SIGNED COMMERCIAL INVOICE IN 3 COPIES INDIC...,1. ALL DOCUMENTS TO BE FORWARDED IN ONE COVER....
18485,LC/02/061/5111,1) BENEFICIARY'S SIGNED COMMERCIAL INVOICE (IN...,+A) PLUS / MINUS 5 PERCENT TOLERANCE IN QUANTI...
2207,M06M12101NS00018,+SIGNED COMMERCIAL INVOICE IN 03 COPIES\r\n+FU...,+NEGOTIATION/PAYMENT UNDER RESERVE OR AGAINST ...
15802,366LCS-67767277,+ COMMERCIAL INVOICE IN 3 ORIGINAL+ FULL SET O...,+ T.T.REIMBURSEMENT : ALLOWED+ IN CASE OF AIR ...
20574,628010783424-M,+ SIGNED INVOICES IN 1 ORIGINAL.\r\n+ PACKING ...,+EACH PRESENTATION CONTAINING DISCREPANCIES WI...
17145,TLGTF2021008031I,1. SIGNED COMMERCIAL INVOICE IN 03 ORIGINALS.\...,1. ALL REQUIRED DOCUMENTS AND ITS ATTACHED LIS...


# 段落切分

In [128]:
x1 = []
x2 = []
for idx in df.index:
    lcno = df.loc[idx,'LCNO']
    x1 = x1 + [ lcno + ' ' + i for i in split_x(str(df.loc[idx,'46A']))]
    x2 = x2 + [ lcno + ' ' + i for i in split_x(str(df.loc[idx,'47A']))]
len(x1),len(x2)

(74569, 113515)

# 執行寶典搜索

In [129]:
from tqdm import tqdm_notebook as tqdm
def Collection_method(x,寶典):
    labels = {}
    for i in tqdm(x):
        products = []
        for p in 寶典:
            if p in i:
                products.append(p)
        try:
            labels[i] = max(products,key=len)
        except:
            labels[i] = 'not find'
    predict = pd.DataFrame(columns=['string_X_train','predict'])
    predict['string_X_train'] = labels.keys()
    predict['predict'] = labels.values()
    return predict

In [130]:
result1 = Collection_method(x1,list(寶典.keys()))
result2 = Collection_method(x2,list(寶典.keys()))
result1['from'] = '46A'
result2['from'] = '47A'
result = result1.append(result2)
result.head()

<ipython-input-129-5296167edb0a>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(x):


,string_X_train,predict,from
0,08761FLC0003220 1. FULL SET OF(3 NEGOTIABLE CO...,BILL OF LADING,46A
1,08761FLC0003220 2.BENEFICIARYS SIGNED COMMERCI...,COMMERCIAL INVOICE,46A
2,08761FLC0003220 3.PACKING LIST..,PACKING LIST,46A
3,08761FLC0003220 4.CERTIFICATE OF ORIGIN IN TWO...,CERTIFICATE OF ORIGIN,46A
4,08761FLC0003220 5.ONE SET OF NON NEGOTIABLE DO...,not find,46A


In [131]:
def k(x):
    return x.split(' ')[0]
def j(x):
    return ' '.join(x.split(' ')[1:])
result['LCNO'] = result['string_X_train'].apply(k)
result['string_X_train'] = result['string_X_train'].apply(j)
result

,string_X_train,predict,from,LCNO
0,1. FULL SET OF(3 NEGOTIABLE COPIES PLUS 3 NON-...,BILL OF LADING,46A,08761FLC0003220
1,2.BENEFICIARYS SIGNED COMMERCIAL INVOICE(S) IN...,COMMERCIAL INVOICE,46A,08761FLC0003220
2,3.PACKING LIST..,PACKING LIST,46A,08761FLC0003220
3,4.CERTIFICATE OF ORIGIN IN TWO COPIES ISSUED B...,CERTIFICATE OF ORIGIN,46A,08761FLC0003220
4,5.ONE SET OF NON NEGOTIABLE DOCUMENTS TO BE SE...,not find,46A,08761FLC0003220
...,...,...,...,...
71308,+AN EXTRA COPY OF INVOICE AND TRANSPORT DOCUME...,TRANSPORT DOCUMENTS,47A,904IUSN203640001
71309,+THE AMOUNT OF EACH DRAWING MUST BE ENDORSED O...,not find,47A,904IUSN203640001
71310,+A PROCESSING FEE OF USD70.00 OR ITS EQUIVALEN...,not find,47A,904IUSN203640001
71311,+DOCUMENTARY PRESENTATION RECEIVED AT OUR COUN...,not find,47A,904IUSN203640001


In [132]:
result['class'] = result['predict'].map(寶典)

In [133]:
result.head(20)

,string_X_train,predict,from,LCNO,class
0,1. FULL SET OF(3 NEGOTIABLE COPIES PLUS 3 NON-...,BILL OF LADING,46A,08761FLC0003220,提單
1,2.BENEFICIARYS SIGNED COMMERCIAL INVOICE(S) IN...,COMMERCIAL INVOICE,46A,08761FLC0003220,商業發票
2,3.PACKING LIST..,PACKING LIST,46A,08761FLC0003220,裝箱單
3,4.CERTIFICATE OF ORIGIN IN TWO COPIES ISSUED B...,CERTIFICATE OF ORIGIN,46A,08761FLC0003220,產地(商會)證明
4,5.ONE SET OF NON NEGOTIABLE DOCUMENTS TO BE SE...,not find,46A,08761FLC0003220,NaN
5,6.CERTIFICATE FROM SHIPPING COMPANY OR ITS AGE...,CERTIFICATE FROM SHIPPING COMPANY,46A,08761FLC0003220,船證
6,7.INSURANCE POLICIES OR CERTIFICATES IN ORIGIN...,INVOICE,46A,08761FLC0003220,商業發票
7,8.CERTIFICATE OF ANALYSIS,not find,46A,08761FLC0003220,NaN
8,1.DRAFTS FOR 100PCT OF INVOICE VALUE..,INVOICE,46A,OLC0005200038327,商業發票
9,2.COMPLETE SET OF CLEAN ON BOARD BILLS OF LADI...,not find,46A,OLC0005200038327,NaN


In [134]:
result.head(1)['string_X_train']

0    1. FULL SET OF(3 NEGOTIABLE COPIES PLUS 3 NON-...
Name: string_X_train, dtype: object

In [135]:
result[result['predict']!='not find'].shape

(49417, 5)

In [136]:
result[result['predict']=='not find'].shape

(67374, 5)

In [137]:
result.to_csv('../data/preprocess_for_SQUAD_押匯文件.csv')